In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')



In [2]:
from config_paper import key_id, secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url )

In [4]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-04-27'

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'400000'

In [7]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [8]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.25

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


In [10]:
returneddata = api.get_barset('FLS',barTimeframe,limit=150).df
returneddata2=api.polygon.historic_agg_v2( 'DISCA', 1, 'minute',_from='2020-04-23', to=startdate,).df
#data_indic=populateindicators(returneddata['AMP'])


In [11]:
populateindicators(returneddata2)

,open,high,low,close,volume,mfi,minusdi,plusdi,rsi,adx
timestamp,,,,,,,,,,
2020-04-23 05:12:00-04:00,20.2400,20.240,20.24,20.24,150,NaN,NaN,NaN,NaN,NaN
2020-04-23 05:13:00-04:00,20.2400,20.240,20.24,20.24,403,NaN,NaN,NaN,NaN,NaN
2020-04-23 06:40:00-04:00,20.1000,20.100,20.05,20.05,1629,NaN,NaN,NaN,NaN,NaN
2020-04-23 06:41:00-04:00,20.0000,20.000,20.00,20.00,100,NaN,NaN,NaN,NaN,NaN
2020-04-23 06:44:00-04:00,20.0500,20.050,20.05,20.05,274,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-04-27 14:06:00-04:00,22.5400,22.545,22.52,22.53,23972,68.941534,14.134850,42.578579,68.192060,37.822373
2020-04-27 14:07:00-04:00,22.5357,22.550,22.53,22.55,30208,72.846424,13.381885,41.642165,70.196456,38.789337
2020-04-27 14:08:00-04:00,22.5500,22.570,22.54,22.57,40836,72.925422,12.321590,43.624946,72.050413,40.015261


In [ ]:
for minu in range(500) :
        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from='2020-04-20', to=startdate,).df
            #api.get_barset(symbol,barTimeframe,limit=150).df


            # Calculated trading indicators
            #populate exponential moving averages
            data=populateindicators(returned_data) 


             # Create the buy signal here
            if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['rsi'][-1] < 35 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] < data['plusdi'][-1]:


                openPosition = api.get_position(symbol)

                # Opens new position if one does not exist
                if openPosition == 0:
                    cashBalance = api.get_account().cash

                    targetPositionSize =   int(int(cashBalance) / (data['close'][-1] / positionSizing))# Calculates required position size

                    returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                    print(returned)


            elif data['adx'][-1] > 22 and \
            data['mfi'][-1] > 63 and \
            data['rsi'][-1] > 74 and \
            data['plusdi'][-1] > 25 and \
            data['minusdi'][-1] < data['plusdi'][-1]:
                # Closes position  SMA50
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,openPosition,"sell","market","day") # Market order to fully close position
                print(returned)
        
        except:
                print(symbol)
                
        #interval should be an integer, the number of seconds to wait
    time.sleep(20)

MMM
MO
BRKB
BFB
FTV
ITW
JCI
KSS
M
MKTK
MYL
NUE
ORCL
PWR
RTN
UTX

ANET
BRKB
BFB
CE
DD
FTV
JCI
MKTK
NUE
RTN
TSCO
UTX

MMM
APA
BRKB
BFB
DISCA
JCI
M
MKTK
